In [ ]:
# default_exp samplers.common

# Samplers Common

> Collection of helper functions and constants common to both Circuit and Protocol samplers

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
#export
import qsam.math as math
from qsam.circuit import partition

import itertools as it
import functools
import numpy as np

# Constants

In [ ]:
#export
ONE_QUBIT_GATES = {'H', 'X', 'Z'}
TWO_QUBIT_GATES = {'CNOT'}

GATE_GROUPS = {'p': ONE_QUBIT_GATES | TWO_QUBIT_GATES,
               'p1': ONE_QUBIT_GATES,
               'p2': TWO_QUBIT_GATES,
               }

# Data structures

In [ ]:
#export
from anytree import RenderTree, NodeMixin
from anytree.exporter import JsonExporter
from anytree.importer import JsonImporter, DictImporter

In [ ]:
#export
class CountNode(NodeMixin):
    def __init__(self, name, parent=None, children=None, counts=0, 
                 ckey=None, is_deterministic=False, is_fail=False):
        super(CountNode, self).__init__()
        
        self.name = name
        self.ckey = ckey
        self.counts = counts
        self.is_deterministic = is_deterministic
        self.is_fail = is_fail
        
        self.parent = parent
        if children:
            self.children = children
            
    @property
    def rate(self):
        assert not self.ckey and not self.is_root
        if self.parent.counts < 2: return 0.5
        else: return self.counts / self.parent.counts

    @property
    def var(self, var_fn=math.Wilson_var):
        assert not self.ckey and not self.is_root
        if self.parent.counts < 2: return 1.0
        else: return var_fn(self.rate, self.parent.counts)
    
    def __str__(self):
        if self.is_root or self.ckey:
            return f'{self.name} ({self.counts})'
        else:
            return f'{self.name} ({self.counts}/{self.parent.counts}, {self.var:.2e})'

In [ ]:
#export
class SampleTree:
    def __init__(self, root=None):
        self.root = root
    
    def add(self, name, parent=None, **kwargs):
        if parent == None:
            if self.root == None:
                self.root = CountNode(name, **kwargs)
            return self.root
        else:
            child_match = [n for n in parent.children if n.name==name]
            if child_match: return child_match[0]
            else: return CountNode(name, parent=parent, **kwargs)
        
    def update(self, name, parent=None, **kwargs):
        node = self.add(name, parent, **kwargs)
        node.counts += 1
        return node
        
    def detach(self, node):
        node.parent = None
        
    def rate(self, const):
        p = 0
        for leaf in self.root.leaves:
            if not leaf.is_root and not leaf.parent.is_deterministic and leaf.is_fail: 
                prod = 1
                for n in leaf.path[1:]:
                    if n.ckey: prod *= const[n.ckey[0]][n.ckey[1]]
                    else: prod *= n.rate
                p += prod
        return p
    
    def var(self, const):
        var = 0
        for node in [n for n in self.root.descendants if not n.ckey]:
            if not node.parent.is_deterministic:
                twig = 1.0
                for n in node.path[1:-1]:
                    if n.ckey: twig *= const[n.ckey[0]][n.ckey[1]]
                    else: twig *= n.rate
                                    
                subtree = 0
                for leaf in node.leaves:
                    if not leaf.is_fail:
                        continue
                    prod = 1
                    for n in leaf.iter_path_reverse():
                        if n == node: break
                        elif n.ckey: prod *= const[n.ckey[0]][n.ckey[1]]
                        else: prod *= n.rate
                    subtree += prod
                var += node.var * twig**2 * subtree**2
        return var
    
    def delta(self, const):
        path_sum = 0.0
        for leaf in self.root.leaves:
            if not leaf.is_root:
                prod = 1.0
                for n in leaf.path[1:]:
                    if n.ckey: prod *= const[n.ckey[0]][n.ckey[1]]
                    else: prod *= n.rate
                path_sum += prod
        return 1.0 - path_sum
    
    def __str__(self):
        return '\n'.join([f'{pre}{node}' for pre, _, node in RenderTree(self.root)])
    
    def save(self, fname):
        exporter = JsonExporter(indent=2, sort_keys=False)
        with open(fname, 'w') as f:
            exporter.write(self.root, f)

    def load(self, fname):
        dictimporter = DictImporter(CountNode)
        importer = JsonImporter(dictimporter)
        with open(fname, 'r') as f:
            self.root = importer.read(f)
        return self

______
# Callback functions

In [ ]:
#export
def curried(func):
    """A decorator that curries the given function.

    @curried
    def a(b, c):
        return (b, c)

    a(c=1)(2)  # returns (2, 1)
    """
    @functools.wraps(func)
    def _curried(*args, **kwargs):
        if 'run' in kwargs.keys(): return func(*args, **kwargs)
        partial_func = functools.partial(func, *args, **kwargs)
        functools.update_wrapper(partial_func, func)
        return partial_func
    return _curried

In [ ]:
#export
def stats(**kwargs):
    sampler = kwargs['sampler']
    return sampler.stats(**kwargs)

In [ ]:
#export
@curried
def store(fn_names, **kwargs):
    sampler = kwargs['sampler']
    if not isinstance(fn_names, (list, tuple, set)): fn_names = [fn_names]
    for fn_name in fn_names:
        assert fn_name in kwargs, f'`{fn_name}` not in `kwargs` {kwargs}'
        cur_store = sampler.store.get(fn_name, [])
        sampler.store[fn_name] = cur_store + [kwargs[fn_name]]
    return None

In [ ]:
#export
@curried
def rel_std_target(target=0.1, **kwargs):
    assert 'stats' in kwargs, f'`stats` not in `kwargs` {kwargs}'
    p_L, std, delta = kwargs['stats']
    if p_L > 0 and (std + delta) / p_L < target: 
        print(f'Rel. std target of {target} reached. Sampling stopped.') 
        return StopIteration

In [ ]:
#export
@curried
def plot_stats(sample_range, **kwargs):
    assert 'stats' in kwargs, f'`stats` not in `kwargs` {kwargs}'
    p_L_low, std, delta = kwargs['stats']
    p_L_up = p_L_low + delta
    
    import matplotlib.pyplot as plt
    if 'ax' not in kwargs: fig,ax = plt.subplots(figsize=(6,4))
    else: ax = kwargs['ax']
    
    if np.array_equal(p_L_up, p_L_low):
        ax.errorbar(sample_range, p_L_low, fmt='--', c="black", yerr=std, label="Direct MC")
    else:
        ax.plot(sample_range, p_L_low, label="SS low")
        ax.fill_between(sample_range, p_L_low-std, p_L_low+std, alpha=0.2)
        ax.plot(sample_range, p_L_up, label="SS up")
        ax.fill_between(sample_range, p_L_up-std, p_L_up+std, alpha=0.2)

    ax.plot(sample_range, sample_range, 'k:', alpha=0.5)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.legend()
    return fig,ax

## Callbacks for Protocol sampler only:

In [ ]:
#export
def verbose(msmt, _node, node, circuit, faults=None, fault_circuit=None, **kwargs):
    pauli_faults = [] if not circuit._noisy else [f'Tick {tick} :: {fault_circuit[tick]}' for tick,_ in faults]
    if _node == 'COR': print(f"Node {_node}, Circuit {circuit} -> {node}")
    else: print(f"Node {_node}, Faults {pauli_faults}, Measured {msmt}-> {node}")

In [ ]:
#export
@curried
def plot_stored_stats(plot_delta=False, labels=['$p_L\pm std$', 'std', '$\delta$'], **kwargs):
    sampler = kwargs['sampler']
    assert 'stats' in sampler.store, f'Sampler {sampler} has no stored stats.'
    stored_stats = np.array(sampler.store['stats'])
    if stored_stats.size == 0: return None
    
    import matplotlib.pyplot as plt
    p_L, std, delta = stored_stats.T
    x = range(len(p_L))
    delta_0 = np.all(delta == 0)
    if delta_0: n_plots = 2
    else: n_plots = 3
    
    fig, ax = plt.subplots(1, n_plots, figsize=(20, 4))
    
    ax[0].plot(x, p_L)
    if plot_delta: ax[0].fill_between(x, p_L - (std + delta), p_L + (std + delta), alpha=0.2)
    else: ax[0].fill_between(x, p_L - std, p_L + std, alpha=0.2)
    ax[1].plot(x, std)
    if not delta_0:
        ax[2].plot(x, delta)
        ax[2].set_yscale('log')
    
    for i in range(n_plots):
        ax[i].set_xlabel('# of samples')
        ax[i].set_ylabel(labels[i])
    return fig,ax

## Callbacks for Subset Protocol sampler only:

In [ ]:
#export
def subset_rates(**kwargs):
    sampler = kwargs['sampler']
    path_rates = []
    fail_rates = []
    path_names = []
    for leaf in sampler.tree.root.leaves:
        if not leaf.is_root and leaf.is_fail:
            path_str = ":".join([f'{n.name}' for n in leaf.path])
            path_names.append(path_str)
            fail_rates.append(leaf.rate)
            prod = 1
            for n in leaf.path[1:]:
                if n.ckey: prod *= sampler.Aws_pmax[n.ckey[0]][n.ckey[1]]
                else: prod *= n.rate
            path_rates.append(prod)
    return (path_names, path_rates, fail_rates)

In [ ]:
#export
def plot_subset_rates(**kwargs):
    sampler = kwargs['sampler']
    assert 'subset_rates' in sampler.store, f'Sampler {sampler} has no stored subset rates.'
    
    names, path_rates, fail_rates = np.array(sampler.store['subset_rates'], dtype=object).T

    
    def fill_na(arr):
        max_len = np.max([len(sub_arr) for sub_arr in arr])
        return np.asarray([np.pad(sub_arr, (0, max_len - len(sub_arr)), constant_values=np.nan) for sub_arr in arr])
    
    path_rates = fill_na(path_rates)
    fail_rates = fill_na(fail_rates)
    
    legend_labels = names[-1]
    n_samples = fail_rates.shape[0]
    
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    for i, rates in enumerate([fail_rates, path_rates]):
        ax[i].plot(range(n_samples), rates)
        ax[i].set_xlim([0, n_samples])
    ax[1].legend(legend_labels, bbox_to_anchor=(1.04, 1), loc='upper left')
    return fig, ax

In [ ]:
#export
def path_prods(**kwargs):
    sampler = kwargs['sampler']
    paths, path_fails, prodAws, prodAwPws_m1, prodAwPws = [], [], [], [], []
    for leaf in sampler.tree.root.leaves:
        if not leaf.is_root:
            prod = 1
            for n in leaf.path[1:]:
                if n.ckey: prod *= sampler.Aws_pmax[n.ckey[0]][n.ckey[1]]
            prodAws.append(prod)
            
            for n in leaf.path[1:-1]:
                if not n.ckey: prod *= n.rate
            prodAwPws_m1.append(prod)
            
            prod *= leaf.rate if leaf.is_fail else 0
            prodAwPws.append(prod)
            
            path_str = ":".join([f'{n.name}' for n in leaf.path])
            paths.append( path_str + f" ({leaf.counts}/{sampler.tree.root.counts})" )
            path_fails.append(leaf.is_fail)
            
    return paths, path_fails, prodAws, prodAwPws, prodAwPws_m1

In [ ]:
#export
def plot_paths(**kwargs):
    assert 'path_prods' in kwargs, f'`path_prods` not in `kwargs` {kwargs}'
    path_prods = kwargs['path_prods']
    
    import matplotlib.pyplot as plt
    
    paths, path_fails, prodAws, prodAwPws, prodAwPws_m1 = path_prods
    sorted_ids = np.argsort(prodAws)[::-1]
    paths = np.array(paths)[sorted_ids]
    path_fails = np.array(path_fails)[sorted_ids]
    prodAws = np.array(prodAws)[sorted_ids]
    prodAwPws = np.array(prodAwPws)[sorted_ids]
    prodAwPws_m1 = np.array(prodAwPws_m1)[sorted_ids]
    
    fig,ax = plt.subplots(figsize=(20,4))
    ax.bar(paths, prodAws, label='$\prod_{n\in P} A_w$')
    ax.bar(paths, prodAwPws_m1, label='$\prod_{n-1\in P} A_{w} p_n$')
    ax.bar(paths, prodAwPws, label='$\prod_{n\in P} A_{w} p_n$')
    ax.set_yscale('log')
    plt.setp(ax.get_xticklabels(), ha="right", rotation=45)
    
    xticklabels = ax.get_xticklabels()
    for i, (path, is_fail) in enumerate(zip(paths,path_fails)):
        if is_fail: plt.setp(xticklabels[i], weight='bold')
            
    ax.legend()
    return fig, ax

In [ ]:
#export
def plot_path_hist(**kwargs):
    sampler = kwargs['sampler']
    
    path_cnts = {}
    for leaf in sampler.tree.root.leaves:
        if not leaf.is_root:
            path_str = ":".join([f'{n.name}' for n in leaf.path[:-1]])
            if path_str in path_cnts.keys():
                cnts, fails = path_cnts[path_str]
                path_cnts[path_str] = (cnts, fails + (leaf.counts if leaf.is_fail else 0))
            else:
                path_cnts[path_str] = (leaf.parent.counts, (leaf.counts if leaf.is_fail else 0))
            
    paths = list(path_cnts.keys())
    cnts, f_cnts = np.array(list(path_cnts.values())).T
    sorted_ids = np.argsort(cnts)[::-1]
    paths = np.array(paths)[sorted_ids]
    cnts = cnts[sorted_ids]
    f_cnts = f_cnts[sorted_ids]
    
    import matplotlib.pyplot as plt
    fig,ax = plt.subplots(figsize=(20,4))
    ax.bar(paths, cnts, label="Path count")
    ax.bar(paths, f_cnts, label="Fail count")
    ax.set_yscale('log')
    plt.setp(ax.get_xticklabels(), ha="right", rotation=45)
    ax.legend()
    return ax, fig

In [ ]:
#export
@curried
def plot_save_tree(fname='temp.png', **kwargs):
    sampler = kwargs['sampler']
    n_samples = sampler.tree.root.counts

    def edgeattrfunc(node, child):
        weight = 10.0 * child.counts / n_samples
        weight = 5.0 if weight > 5.0 else weight
        return f'penwidth="{weight}"'

    def nodeattrfunc(node):
        node_form = ""
        if node.is_fail: 
            node_form = ', style=filled, fillcolor="#ff9999"'
        return f'label="{node.name}"' + node_form

    from anytree.exporter import UniqueDotExporter
    from IPython.display import Image, display ## ! requirements: jupyter, graphviz
    
    img = UniqueDotExporter(sampler.tree.root, 
                      edgeattrfunc=edgeattrfunc,
                      nodeattrfunc=nodeattrfunc).to_picture(fname)
    display(Image(fname))
    return fname

_____
# Other functions

## Subset analytics

In [ ]:
#export
def circuit_partitions(circ, partition_names):
    return [partition(circ, GATE_GROUPS[name]) for name in partition_names]

In [ ]:
#export
def circuit_weight_vectors(w_max, w_exclude={}):
    w_exclude = set((w,) if isinstance(w, int) else w for w in w_exclude)
    w_maxs = [w_max] if isinstance(w_max, int) else w_max

    w_upto_w_maxs = [tuple(range(w_max+1)) for w_max in w_maxs]
    w_vecs = list(it.product( *w_upto_w_maxs ))
    filtered_w_vecs = [w for w in w_vecs if w not in w_exclude]
    return filtered_w_vecs

In [ ]:
#export
def circuit_subset_occurence(partitions, partition_w_vecs, p_phy_per_partition):
    """Return (weight)x(p_phys) (parition) subset occurance matrix transforming p_SS vector to p_L vector"""
    n_partition_elems = np.array([len(p) for p in partitions])
    Aws = np.array([math.binom(w_vec, n_partition_elems, p_phy_per_partition) for w_vec in partition_w_vecs])
    Aws = np.product(Aws, axis=-1) # mult Aws for multi-parameter, i.e. multi-partitions
    return Aws

In [ ]:
#export
def protocol_partitions(circuits_dict, partition_names):
    return {c_hash: circuit_partitions(circ, partition_names)
           for c_hash, circ in circuits_dict.items() if circ._noisy}

In [ ]:
#export
def protocol_weight_vectors(partition_dict):
    return {c_hash: circuit_weight_vectors([len(p) for p in partitions]) 
            for c_hash, partitions in partition_dict.items()}

In [ ]:
#export
def protocol_subset_occurence(partition_dict, w_vecs_dict, p_phys):
    return {c_hash: circuit_subset_occurence(partition_dict[c_hash], w_vecs, p_phys)
            for c_hash, w_vecs in w_vecs_dict.items()}

## Subset filtering and selection

In [ ]:
#export
def w_plus1_filter(sampler, w_ids, circuit_hash, **kwargs):
    n_Aws = np.ma.array(sampler.Aws_pmax[circuit_hash])
    n_Aws[w_ids,] = np.ma.masked # mask existing (already sampled) subsets
    w_ids.append(np.argmax(n_Aws)) # add next most important subset as possible candidate
    return w_ids

In [ ]:
#export
def random_sel(w_ids, **kwargs):
    return np.random.choice(w_ids)

In [ ]:
#export
def ERV_sel(self, sampler, w_ids, circuit_hash, tree_node, **kwargs):
    erv_deltas = []
    v_L = sampler.tree.var(sampler.Aws_pmax)
    delta = sampler.tree.delta(sampler.Aws_pmax)
    for idx in w_ids:
        w_vec = sampler.w_vecs[circuit_hash][idx]

        _tree_node = sampler.tree.add(w_vec, parent=tree_node, ckey=(circuit_hash, idx))
        __tree_node = sampler.tree.add('FAIL', parent=_tree_node, is_fail=True)
        _delta = sampler.tree.delta(sampler.Aws_pmax)
        _rate = __tree_node.rate

        _tree_node.counts += 1
        v_L_minus = sampler.tree.var(sampler.Aws_pmax)

        __tree_node.counts += 1
        v_L_plus = sampler.tree.var(sampler.Aws_pmax)

        _v_L = _rate * v_L_plus + (1 - _rate) * v_L_minus
        erv_delta = np.abs(v_L - _v_L) + (delta - _delta)
        erv_deltas.append( erv_delta )

        # revert the change
        _tree_node.counts -= 1
        __tree_node.counts -= 1
        if _tree_node.counts == 0: sampler.tree.detach(_tree_node)
        if __tree_node.counts == 0: sampler.tree.detach(__tree_node)

    idx = np.argmax(erv_deltas)
    return w_ids[idx]